# Data cleaning projects

In [128]:
import pandas as pd

In [129]:
listing = pd.read_csv(r'C:\Users\tamla_f1yewqh\OneDrive\Desktop\Python\Projects\Airbnb\Dataload\listings.csv')

C:\Users\tamla_f1yewqh\AppData\Local\Temp\ipykernel_18312\1654123833.py:1: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  listing = pd.read_csv(r'C:\Users\tamla_f1yewqh\OneDrive\Desktop\Python\Projects\Airbnb\Dataload\listings.csv')


## Clean listing data

### Setting requirements

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

- [x] get rid of unnecessary columns
  
A- Price factors:
  - [x] facitlity: property_type, room_type, accomodates, bathrooms, bathrooms text, bedrooms, beds, amenities, price
  - [x] location: latitde, longitude, neighbourhood_cleanse,
  - [x] time: calendar factors
  

B- Occupancy rate: 
  - [x] location, time, price factors
  - [x] occupancy metrics: availability, minimum/ maximum night, 
  - [x] host factors: check host_location, convert host_since/ response time/ response rate/ acceptance rate/ superhost to suitable data types
  - [ ] review related: keep all review column in listing, and count the number of review for each listing in review data

C- Other:
  - [x] Outliers check
  - [x] Handle null values

In [130]:
col_to_drop = ['Unnamed: 0','neighbourhood_group_cleansed', 'listing_url', 'scrape_id', 
                'last_scraped','source', 'description','neighborhood_overview', 'picture_url',
                'host_url','host_thumbnail_url','host_picture_url', 'host_neighbourhood', 
                'host_listings_count', 'host_total_listings_count', 'neighbourhood', 
                'minimum_minimum_nights', 'maximum_minimum_nights','minimum_maximum_nights', 
                'maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm', 
                'calendar_updated', 'calendar_last_scraped','first_review',
                'last_review','license', 'calculated_host_listings_count',
                'calculated_host_listings_count_entire_homes',
                'calculated_host_listings_count_private_rooms', 
                'calculated_host_listings_count_shared_rooms']

In [131]:
df_listing = listing.drop(col_to_drop, axis = 1)

In [ ]:
df_listing.info()

### Clean data related to price factors

In [132]:
#checking format of property_type, room_type, accommodates
df_listing.groupby('room_type').count()['id']

room_type
Entire home/apt    14736
Hotel room           458
Private room        7931
Shared room          526
Name: id, dtype: int64

In [ ]:
df_listing.groupby('property_type').count()['id']

In [134]:
df_listing.groupby('accommodates').count()['id'].head(5)

accommodates
1      840
2    13634
3     2685
4     3327
5      744
Name: id, dtype: int64

In [135]:
# bathrooms na values
df_listing['bathrooms'].isna().sum()

4642

In [136]:
# checking if the bathrooms text can fill the na in bathrooms
len(df_listing[df_listing['bathrooms'].isna()][['bathrooms','bathrooms_text']])

4642

In [137]:
# compared null value with bathroom_text
df_listing[df_listing['bathrooms'].isna()][['bathrooms','bathrooms_text']].head(5)

,bathrooms,bathrooms_text
1,NaN,1 bath
2,NaN,1 bath
8,NaN,1 bath
15,NaN,4 baths
17,NaN,1 bath


In [138]:
# testing samples
sample = pd.DataFrame(['1.5','1.0', '1 bath','1.5 shared bath'], columns=['bathrooms'])
sample['bath'] = sample['bathrooms'].str.split(' ',expand=True)[0]
sample

,bathrooms,bath
0,1.5,1.5
1,1.0,1.0
2,1 bath,1
3,1.5 shared bath,1.5


In [139]:
#convert value on bathrooms text to number
df_listing['bathrooms_text'] = df_listing['bathrooms_text'].str.split(' ',expand=True)[0]

In [140]:
# fillna in bathrooms with bathrooms text value
df_listing['bathrooms'] = df_listing['bathrooms'].fillna(df_listing['bathrooms_text'])

In [141]:
# clean and transform type of bathrooms column
df_listing['bathrooms'] = pd.to_numeric(df_listing['bathrooms'], errors='coerce')

In [142]:
# check the how many na value left
df_listing['bathrooms'].isna().sum()

85

In [ ]:
# checking amenities
df_listing['amenities'].sample(10)

In [144]:
# clean format
df_listing['amenities'] = df_listing['amenities'].apply(lambda x: x.replace('[','').replace(']','').replace('"','').replace(' ,',',').replace(', ',',').split(','))

In [145]:
# adding the number of amenities
df_listing['amenities_count'] = df_listing['amenities'].apply(len)

In [ ]:
# clean price column. The currency is local currency (baht)
df_listing['price'].head(10)

In [147]:
df_listing['price_baht'] = df_listing['price'].str.replace('$','')
df_listing['price_baht'] = df_listing['price_baht'].str.replace(',','')
df_listing['price_baht'] = df_listing['price_baht'].str.replace('.00','')
df_listing['price_baht'] = pd.to_numeric(df_listing['price_baht'])

### Clean location/host/review factors

In [ ]:
# checking neighborhood format. The format seem to be good, no need further edit
df_listing.groupby('neighbourhood_cleansed').count()['id']

In [ ]:
# host factors: host_since convert to datetime, check lost_location, convert response time/ response rate/ acceptance rate/ superhost to suitable data types
df_listing.groupby('host_location').count()['id']

In [150]:
# how many host are located in Bangkok
len(df_listing[df_listing['host_location'].str.contains(r'Bangkok') == True])

14626

In [151]:
# convert the host since into date time
df_listing['host_since'] = pd.to_datetime(df_listing['host_since'], errors='coerce')

In [ ]:
# checking host_response time
df_listing.groupby('host_response_time').count()['id']

In [153]:
# checking response rate: remove % sign and convert to numeric
df_listing['host_response_percent'] = pd.to_numeric(df_listing['host_response_rate'].str.replace('%',''))

In [154]:
# checking host acceptance rate
df_listing['host_acceptance_percent'] = pd.to_numeric(df_listing['host_acceptance_rate'].str.replace('%',''))

In [ ]:
#convert boolean type 
df_listing['host_is_superhost'].astype('bool')
df_listing['host_identity_verified'].astype('bool')
df_listing['host_has_profile_pic'].astype('bool')
df_listing['instant_bookable'].astype('bool')
df_listing['host_has_profile_pic'].astype('bool')

In [ ]:
# recheck columns and drop unncessary columns
df_listing.info()

In [157]:
col_to_drop2 = ['host_location', 'host_about', 'host_response_time', 'host_acceptance_rate', 'bathrooms_text', 'price','host_response_rate','host_verifications']

In [158]:
df_listing = df_listing.drop(col_to_drop2, axis =1)

In [ ]:
df_listing.info()

### Handle missing values/ outliers

In [ ]:
# outliers detection: accommodates, rooms, beds, price 
df_listing.groupby('accommodates').count()['id'] # there seem no outliers in range of accomodates

In [ ]:
# checking outliers for bedrooms
df_listing.groupby('bedrooms').count()['id'].sort_values(ascending=False) 

In [ ]:
# check what kind of listing has 0 bedrooms.
df_listing[df_listing['bedrooms'] == 0][['beds','bedrooms','property_type']].sample(10)

In [163]:
# check 0 bedrooms case and compare to beds not 0
len(df_listing[(df_listing['bedrooms'] == 0) & (df_listing['beds'] != 0)])

872

In [164]:
# question: should the 0 bedrooms value be fill with the beds value assuming 1 bed = 1 bedroom. If so, what is the average bed per bedroom
bed_num = df_listing[(df_listing['beds'] != 0) & (df_listing['bedrooms'] != 0)]['beds'].sum()
bedroom_num = df_listing[(df_listing['beds'] != 0) & (df_listing['bedrooms'] != 0)]['bedrooms'].sum()
bed_num/bedroom_num 

1.069291448031414

In [165]:
# since the average beds per bedroom is nearly 01, the 0 number of bedrooms col can get replace with non-zero in beds col
df_listing['bedrooms'] = df_listing.apply(lambda x: x['beds'] if x['bedrooms'] == 0 else x['bedrooms'], axis = 1 )

In [166]:
# check the number of 0 bedrooms left
len(df_listing[df_listing['bedrooms'] == 0])

60

In [167]:
# number of listing with null price (does not list their price)
df_listing['price_baht'].isna().sum()

4639

In [169]:
# checking samples the null-price listing if there is any bookings, reviews, ... If there is like no activity, maybe these listings are no longer in business
df_listing[df_listing['price_baht'].isna()][['availability_30','availability_60','availability_90','availability_365','host_acceptance_percent']].sample(10)

,availability_30,availability_60,availability_90,availability_365,host_acceptance_percent
213,0,0,0,0,NaN
11808,0,0,0,0,NaN
11691,0,0,0,0,99.0
16766,30,60,90,270,96.0
4811,12,42,72,73,NaN
2254,0,0,0,0,NaN
3699,0,0,0,0,NaN
7484,0,0,0,0,NaN
9325,0,0,20,284,70.0
1776,28,58,88,88,0.0


In [170]:
# checking how many of null-price listings has 0 in availability, 0 reviews and null in acceptance rate
row_to_drop = df_listing[(df_listing['price_baht'].isna()) & (df_listing['availability_365'] == 0) & (df_listing['number_of_reviews']==0) & (df_listing['host_acceptance_percent'].isna())].index
len(row_to_drop)

1767

In [171]:
# drop these rows as these listing does not seem to be in operation
df_listing = df_listing.drop(index = row_to_drop)

In [173]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21884 entries, 0 to 23650
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           21884 non-null  int64         
 1   name                         21884 non-null  object        
 2   host_id                      21884 non-null  int64         
 3   host_name                    21883 non-null  object        
 4   host_since                   21884 non-null  datetime64[ns]
 5   host_is_superhost            20888 non-null  object        
 6   host_has_profile_pic         21884 non-null  object        
 7   host_identity_verified       21884 non-null  object        
 8   neighbourhood_cleansed       21884 non-null  object        
 9   latitude                     21884 non-null  float64       
 10  longitude                    21884 non-null  float64       
 11  property_type                21884 non-null  o

### Save clean file

In [172]:
df_listing.to_csv(r'Dataload_clean/listing_cleaned.csv', index = False)

## Clean calendar and reviews data

### Loading and checking format

In [108]:
import pyarrow as pa

In [109]:
review = pd.read_parquet(r'C:\Users\tamla_f1yewqh\OneDrive\Desktop\Python\Projects\Airbnb\Dataload\reviews.parquet')

In [110]:
calendar = pd.read_parquet(r'C:\Users\tamla_f1yewqh\OneDrive\Desktop\Python\Projects\Airbnb\Dataload\calendar.parquet')

In [ ]:
review.head()

In [ ]:
review.info()

In [37]:
calendar.head()

,column0,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,0,55681,2024-06-27,False,"$1,105.00",None,2.0,365.0
1,1,55681,2024-06-28,False,"$1,105.00",None,2.0,365.0
2,2,55681,2024-06-29,False,"$1,105.00",None,2.0,365.0
3,3,55681,2024-06-30,False,"$1,105.00",None,2.0,365.0
4,4,55681,2024-07-01,False,"$1,105.00",None,2.0,365.0


In [ ]:
calendar.info()

In [114]:
df_review = review.copy()

In [113]:
df_calendar = calendar.copy()

### Setting requirements

A - For review table:
- [x] drop column0
- [x] convert date into datetime format

B - For calendar table:
- [x] drop column0
- [x] convert price into price_baht
- [x] convert date into datetime format
- [x] clean adjusted-price

C - Other:
- [ ] Remove those lines that the listings id are not in listings table (as some listing has been removed at the last step of cleaning listings)

### Cleaning process

In [115]:
# drop column0 in both table
df_review = df_review.drop('column0', axis=1)

In [116]:
df_calendar = df_calendar.drop('column0', axis=1)

In [117]:
# convert datetime format in both table
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_review['date'] = pd.to_datetime(df_review['date'])

In [118]:
# conver the price in calendar to local currency
df_calendar['price_baht'] = df_calendar['price'].apply(lambda x: x.replace('$','').replace(',','').replace('.00',''))

In [119]:
df_calendar['price_baht'] = pd.to_numeric(df_calendar['price_baht'])

In [120]:
df_calendar = df_calendar.drop('price',axis = 1)

In [175]:
# checking both table
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438958 entries, 0 to 438957
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   listing_id     438958 non-null  int64         
 1   id             438958 non-null  int64         
 2   date           438958 non-null  datetime64[ns]
 3   reviewer_id    438958 non-null  int64         
 4   reviewer_name  438953 non-null  object        
 5   comments       438901 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 20.1+ MB


In [122]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8632539 entries, 0 to 8632538
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   listing_id      int64         
 1   date            datetime64[ns]
 2   available       bool          
 3   adjusted_price  object        
 4   minimum_nights  float64       
 5   maximum_nights  float64       
 6   price_baht      float64       
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 403.4+ MB


In [123]:
# check adjusted price in calendar. The adjusted price has mostly null value which is not quite useful, hence it will be droped
df_calendar['adjusted_price'].isna().sum()

8632174

In [124]:
df_calendar = df_calendar.drop('adjusted_price', axis= 1)

In [174]:
# clean out listing id that has been removed in listing data
df_calendar['listing_to_remove'] = df_calendar['listing_id'].isin(df_listing['id'])

In [176]:
df_review['listing_to_remove'] = df_review['listing_id'].isin(df_listing['id'])

In [178]:
df_review = df_review[df_review['listing_to_remove']]

In [180]:
df_calendar = df_calendar[df_calendar['listing_to_remove']]

In [181]:
# drop the listing_to_remove in both table
df_calendar = df_calendar.drop(columns=['listing_to_remove'])
df_review = df_review.drop(columns=['listing_to_remove'])

In [185]:
# check unique value of listing in both table
len(df_review['listing_id'].unique())

15428

In [186]:
# the number of listings on calendar now match the number of listings on listing table
len(df_calendar['listing_id'].unique())

21884

### Save clean files

In [182]:
df_review.to_parquet(r'Dataload_clean/review_clean.parquet', index = False)

In [183]:
df_calendar.to_parquet(r'Dataload_clean/calendar_clean.parquet', index = False)